In [1]:
import numpy as np
import tensorflow as tf
import sys
import time
from datetime import timedelta
import tensorflow.contrib.keras as kr
from sklearn import metrics
from sklearn.model_selection import KFold

import moxing as mox
mox.file.shift('os', 'mox')

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2


In [2]:
trainDataPath = "s3://corpus-2/dataset/corpus_5_new.txt"
vocabPath = "s3://corpus-text-classification1/data/glove.6B.100d.txt"

In [3]:
split_info = {
    "random": False,
    "expert": [20, 4],
    "bundle": [920, 1],
    "table": [37, 3]
}


def dataset_split(info):
    if info:
        [num, pi] = info
        train_data = [[] for i in range(num)]
        with open(trainDataPath, "r", encoding='utf-8') as fp:
            for line in fp.readlines():
                word = line.split()
                info = word[0].split(":")
                index = int(info[pi]) - 1
                label = int(info[0])
                content = word[1:]
                train_data[index].append([content,label])

        for i in range(num):
            np.random.shuffle(train_data[i])
            train_data[i] = np.asarray(train_data[i])

        np.random.shuffle(train_data)   
        return train_data
    
    
    train_data = []
    with open(trainDataPath, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            word = line.split()
            label = int(word[0].split(":")[0])
            content = word[1:]
            train_data.append([content,label])
    
    np.random.shuffle(train_data)
    return np.asarray(train_data)


def mergeData(data_x, data_y):
    merge_x = data_x[0]
    merge_y = data_y[0]
    for i in range(1,len(data_x)):
        merge_x = np.r_[merge_x,data_x[i]]
        merge_y = np.r_[merge_y,data_y[i]]
        
    return merge_x, merge_y


def train_split_data(train_data, split_type):
    
    print(split_type)
    
    test_acc = []
    fold_id = 0
    
    if split_type != "random":
        tx = []
        ty = []
        for ti in train_data:
            x_train, y_train = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
            tx.append(x_train)
            ty.append(y_train)

        tx = np.asarray(tx)
        ty = np.asarray(ty)

        print(len(tx),len(tx[0]),len(tx[1]),len(tx[0][0]))
        
        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            train_x, train_y = mergeData(tx[train_i],ty[train_i])
            test_x, test_y = mergeData(tx[test_i],ty[test_i])
            test_acc.append(classifier.train(
                X_train=train_x,
                y_train=train_y,
                X_eval=test_x,
                y_eval=test_y,
                categories=categories,
                epochs=50
            ))
        
    else:
        tx, ty = process_file(train_data[:,0], train_data[:,1], word_to_id, num_classes, seq_length)
        print(len(tx),len(tx[0]),len(tx[1]))

        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            test_acc.append(classifier.train(
                X_train=tx[train_i],
                y_train=ty[train_i],
                X_eval=tx[test_i],
                y_eval=ty[test_i],
                categories=categories,
                epochs=50
            ))
        
    print(test_acc)
    print("%s, %s, %s, %s" % (np.mean(test_acc),np.std(test_acc),np.std(test_acc,ddof=1),np.var(test_acc)))
    return test_acc

In [4]:
def loadGloVe(filename):
    vocab = []
    embd = []
    print('Loading GloVe!')
    # vocab.append('unk') #装载不认识的词
    # embd.append([0] * emb_size) #这个emb_size可能需要指定
    file = open(filename,'r',encoding='utf-8')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append([float(ei) for ei in row[1:]])
    file.close()
    print('Completed!')
    return vocab,embd


def process_file(contents, labels, word_to_id, num_classes, pad_max_length):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    # contents, labels = readfile(filePath)
    data_id, label_id = [], []
    # 将文本内容转换为对应的id形式
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(labels[i] - 1)  # label_id.append(cat_to_id[labels[i]])
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
    ''' https://blog.csdn.net/TH_NUM/article/details/80904900
    pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        sequences：浮点数或整数构成的两层嵌套列表
        maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
        dtype：返回的numpy array的数据类型
        padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
        truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
        value：浮点数，此值将在填充时代替默认的填充值0
    '''
    y_pad = kr.utils.to_categorical(label_id, num_classes=num_classes)  # 将标签转换为one-hot表示
    ''' https://blog.csdn.net/nima1994/article/details/82468965
    to_categorical(y, num_classes=None, dtype='float32')
        将整型标签转为onehot。y为int数组，num_classes为标签类别总数，大于max(y)（标签从0开始的）。
        返回：如果num_classes=None，返回len(y) * [max(y)+1]（维度，m*n表示m行n列矩阵，下同），否则为len(y) * num_classes。
    '''
    return x_pad, y_pad

In [5]:
categories = ['Retrieve Value', 'Filter', 'Compute Derived Value', 'Find Extremum', 'Sort', 
                  'Determine Range', 'Characterize Distribution', 'Find Anomalies', 'Cluster', 'Correlate']
num_classes = len(categories)

vocab, embd = loadGloVe(vocabPath)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)
word_to_id = dict(zip(vocab, range(vocab_size)))

print(len(embedding),embedding_dim,vocab_size)
 
seq_length = 41  # seq_length = 37  TREC

Loading GloVe!
Completed!
400000 100 400000


In [6]:
class Classifier:

    def __init__(self, model, input_length, output_length):
        self.model = model
        self.input_length = input_length
        self.output_length = output_length

    def compile(self, batch_size=32):
        self._ds_x = tf.placeholder(tf.float32, [None, self.input_length])
        self._ds_y = tf.placeholder(tf.float32, [None, self.output_length])

        ds = tf.data.Dataset.from_tensor_slices((self._ds_x, self._ds_y))
        ds = ds.batch(batch_size)

        self._ds_it = ds.make_initializable_iterator()
        self._input, self._labels = self._ds_it.get_next()

        self._features = self.model(self._input)
        self._output = _create_dense_layer(self._features, self.output_length)

        self._create_acc_computations()
        self._create_backpropagation()

    def _create_acc_computations(self):
        self._predictions = tf.argmax(self._output, 1)
        labels = tf.argmax(self._labels, 1)
        self._accuracy = tf.reduce_mean(
            tf.cast(tf.equal(self._predictions, labels), 'float32'))

    def _create_backpropagation(self):
        losses = tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=self._output,
            labels=self._labels)
        self._loss = tf.reduce_mean(losses)

        optimizer = tf.train.AdamOptimizer(0.001)
        global_step = tf.Variable(0, name="global_step", trainable=False)
        grads_and_vars = optimizer.compute_gradients(self._loss)

        self._train_op = optimizer.apply_gradients(
            grads_and_vars, global_step=global_step)

    def summary(self):
        print('input:', self._input.shape)
        self.model.summary()
        print('output:', self._output.shape)

    def train(self, X_train, y_train, X_eval, y_eval, categories, epochs=20, require_improve=3):
        
        session = tf.Session()
        session.run(tf.global_variables_initializer())
        session.run(tf.local_variables_initializer())
        
        best_vac_acc = 0.0
        last_improved = 0
        
        for e in range(epochs):
            start_time = time.time()
            loss, acc = self._train(X_train, y_train, session)
            duration = time.time() - start_time

            val_loss, val_acc = self._eval(X_eval, y_eval, session)
            
            if val_acc > best_vac_acc:
                best_vac_acc = val_acc
                last_improved = e
                improved_str = '*'
            else:
                improved_str = ''
            
            output = 'Epoch: {:>1}, Train Loss: {:>6.4}, Train Acc: {:>6.2%}, Val Loss: {:>6.4}, Val Acc: {:>6.2%}, Time: {:.2f}s {}'
            print(output.format(e + 1, loss, acc, val_loss, val_acc, duration, improved_str))
            
            if e - last_improved > require_improve:
                print("No optimization for a long time, auto-stopping...")
                
                y_test_cls = np.argmax(y_eval, 1)  # 获得类别
                y_test_pred_cls = np.argmax(self.predict(X_eval, session), 1)
                accuracy_score = metrics.accuracy_score(y_test_cls, y_test_pred_cls)
                
                # evaluate
                print("Precision, Recall and F1-Score...")
                print(metrics.classification_report(y_test_cls, y_test_pred_cls, target_names=categories))
                '''
                sklearn中的classification_report函数用于显示主要分类指标的文本报告．在报告中显示每个类的精确度，召回率，F1值等信息。
                    y_true：1维数组，或标签指示器数组/稀疏矩阵，目标值。 
                    y_pred：1维数组，或标签指示器数组/稀疏矩阵，分类器返回的估计值。 
                    labels：array，shape = [n_labels]，报表中包含的标签索引的可选列表。 
                    target_names：字符串列表，与标签匹配的可选显示名称（相同顺序）。 
                    原文链接：https://blog.csdn.net/akadiao/article/details/78788864
                '''

                print("Confusion Matrix...")
                print(metrics.confusion_matrix(y_test_cls, y_test_pred_cls))
                '''
                混淆矩阵是机器学习中总结分类模型预测结果的情形分析表，以矩阵形式将数据集中的记录按照真实的类别与分类模型作出的分类判断两个标准进行汇总。
                这个名字来源于它可以非常容易的表明多个类别是否有混淆（也就是一个class被预测成另一个class）
                https://blog.csdn.net/u011734144/article/details/80277225
                '''
                break
        # endfor
        session.close()
        return accuracy_score

    def _train(self, X_train, y_train, session):
        import numpy as np

        session.run(
            fetches=self._ds_it.initializer,
            feed_dict={
                self._ds_x: X_train,
                self._ds_y: y_train
            })
        loss, acc, = [], []
        while True:
            try:
                _, vloss, vacc = session.run(
                    fetches=[self._train_op, self._loss, self._accuracy])

                loss.append(vloss)
                acc.append(vacc)
            except tf.errors.OutOfRangeError:
                break
        # endwhile

        loss, acc = np.mean(loss), np.mean(acc)
        return loss, acc

    def _eval(self, X_val, y_val, session):
        session.run(
            fetches=self._ds_it.initializer,
            feed_dict={
                self._ds_x: X_val,
                self._ds_y: y_val
            })

        loss, acc, = 0, 0
        while True:
            try:
                l, vloss, vacc = session.run(
                    fetches=[self._labels, self._loss, self._accuracy])

                loss += vloss * len(l)
                acc += vacc * len(l)
            except tf.errors.OutOfRangeError:
                break

        return loss / len(X_val), acc / len(X_val)

    def predict(self, X, session):
        session.run(self._ds_it.initializer,
                         feed_dict={
                             self._ds_x: X,
                             self._ds_y: np.empty((len(X), self.output_length))
                         }
                         )

        pred = list()
        while True:
            try:
                ppred = session.run(tf.nn.softmax(self._output))

                pred.extend(map(lambda l: l.tolist(), ppred))
            except tf.errors.OutOfRangeError:
                break

        return pred

def _create_dense_layer(x, output_length):
    '''Creates a dense layer
    '''
    input_size = x.shape[1].value
    W = tf.Variable(
        initial_value=tf.truncated_normal(
            shape=[input_size, output_length],
            stddev=0.1))
    b = tf.Variable(
        initial_value=tf.truncated_normal(
            shape=[output_length]))

    dense = tf.nn.xw_plus_b(x, W, b)

    return dense

In [7]:
class ZhouCLSTMModel:
    '''
    Implementation proposal of: https://arxiv.org/pdf/1511.08630
    '''
    def __init__(self, embedding,
        conv_size    = 3,
        conv_filters = 300,
        drop_rate    = 0.5,
        lstm_units   = 150):
        '''Constructor.
        # Parameters:
        conv_size: Size of the convolutions. Number of words that takes each
            convolution step.
        conv_filters: Number of convolution filters.
        drop_rate: Drop rate for the final output of the LSTM layer.
        lstm_units: Size of the states of the LSTM layer.
        '''
        self._embedding    = embedding
        self._conv_size    = conv_size
        self._conv_filters = conv_filters
        self._drop_rate    = drop_rate
        self._lstm_units   = lstm_units

    def __call__(self, input):
        self._embedding_tf = self._create_embedding_layer(
            self._embedding, input)
        
        self._convolution_tf = self._create_convolutional_layers(
            self._conv_size,
            self._conv_filters,
            self._drop_rate,
            self._embedding_tf)
        
        self._lstm_tf = self._create_lstm_layer(
            self._lstm_units,
            self._convolution_tf)
        
        return self._lstm_tf

    def summary(self):
        print('embedding:', str(self._embedding_tf.shape))
        print('conv:', str(self._convolution_tf.shape))
        print('lstm:', str(self._lstm_tf.shape))

    def _create_embedding_layer(self, embedding, input_x):
        embedding = tf.Variable(initial_value=embedding)

        embedded_chars = tf.nn.embedding_lookup(
            embedding, tf.cast(input_x, 'int32'))

        return embedded_chars

    def _create_convolutional_layers(self,
        conv_size, num_filters, drop_rate, embedding):
        
        filter_height = conv_size
        filter_width = embedding.shape[2].value

        filter_shape = [filter_height, filter_width, 1, num_filters]

        W = tf.Variable(
            initial_value=tf.truncated_normal(
                shape=filter_shape,
                stddev=0.1))
        b = tf.Variable(
            initial_value=tf.truncated_normal(
                shape=[num_filters]))

        embedding_expanded = tf.expand_dims(embedding, -1)
        conv = tf.nn.conv2d(
            input=embedding_expanded,
            filter=W,
            strides=[1,1,1,1],
            padding='VALID')
        conv_reduced = tf.reshape(
            tensor=conv,
            shape=[-1, conv.shape[1], conv.shape[3]])

        bias = tf.nn.bias_add(conv_reduced, b)
        c = tf.nn.relu(bias)

        d = tf.nn.dropout(c, keep_prob=drop_rate)
        return d

    def _create_lstm_layer(self, lstm_units, conv_input):
        lstm_cell = tf.nn.rnn_cell.LSTMCell(lstm_units)
        sequence = tf.unstack(conv_input, axis=1)
        (_, (h, _)) = tf.nn.static_rnn(lstm_cell, sequence, dtype=tf.float32)

        return h

In [8]:
word_vector = embedding.astype('float32')
model = ZhouCLSTMModel(embedding=word_vector)

classifier = Classifier(
    model=model,
    input_length=seq_length,
    output_length=num_classes)

classifier.compile(batch_size=32)
classifier.summary()

input: (?, 41)
embedding: (?, 41, 100)
conv: (?, 39, 300)
lstm: (?, 150)
output: (?, 10)


In [ ]:
split_info = {
    # "random": False,
    "expert": [20, 4],
    "bundle": [920, 1],
    "table": [37, 3]
}

kf = KFold(n_splits=10)
test_acc_split = []
for split_type,info in split_info.items():
    train_data = dataset_split(info)
    test_acc_split.append(train_split_data(train_data, split_type))

expert
20 1103 636 41
Fold:  1
Epoch: 1, Train Loss:   1.46, Train Acc: 50.63%, Val Loss:  1.548, Val Acc: 47.38%, Time: 10.06s *
Epoch: 2, Train Loss: 0.6433, Train Acc: 79.59%, Val Loss:  1.607, Val Acc: 57.10%, Time: 7.33s *
Epoch: 3, Train Loss: 0.3473, Train Acc: 89.43%, Val Loss:  1.596, Val Acc: 61.07%, Time: 7.24s *
Epoch: 4, Train Loss: 0.2163, Train Acc: 93.25%, Val Loss:  1.492, Val Acc: 63.77%, Time: 7.20s *
Epoch: 5, Train Loss: 0.1458, Train Acc: 95.45%, Val Loss:  2.043, Val Acc: 59.34%, Time: 7.37s 
Epoch: 6, Train Loss:  0.116, Train Acc: 96.40%, Val Loss:  1.892, Val Acc: 61.53%, Time: 7.16s 
Epoch: 7, Train Loss: 0.08493, Train Acc: 97.32%, Val Loss:  1.924, Val Acc: 65.50%, Time: 7.18s *
Epoch: 8, Train Loss: 0.06725, Train Acc: 97.73%, Val Loss:   2.18, Val Acc: 62.97%, Time: 7.28s 
Epoch: 9, Train Loss: 0.05486, Train Acc: 98.15%, Val Loss:  2.461, Val Acc: 63.83%, Time: 7.15s 
Epoch: 10, Train Loss: 0.04314, Train Acc: 98.45%, Val Loss:  2.275, Val Acc: 64.23%, T

Epoch: 2, Train Loss: 0.6413, Train Acc: 79.74%, Val Loss:  1.047, Val Acc: 70.39%, Time: 7.48s *
Epoch: 3, Train Loss: 0.3473, Train Acc: 88.89%, Val Loss:  1.128, Val Acc: 71.55%, Time: 7.39s *
Epoch: 4, Train Loss: 0.2152, Train Acc: 93.38%, Val Loss:  1.121, Val Acc: 73.94%, Time: 7.52s *
Epoch: 5, Train Loss: 0.1382, Train Acc: 95.73%, Val Loss:  1.158, Val Acc: 75.71%, Time: 7.51s *
Epoch: 6, Train Loss: 0.1039, Train Acc: 96.76%, Val Loss:  1.183, Val Acc: 76.18%, Time: 7.60s *
Epoch: 7, Train Loss: 0.08231, Train Acc: 97.48%, Val Loss:   1.31, Val Acc: 72.78%, Time: 7.51s 
Epoch: 8, Train Loss: 0.06468, Train Acc: 97.88%, Val Loss:  1.243, Val Acc: 76.48%, Time: 7.45s *
Epoch: 9, Train Loss: 0.0546, Train Acc: 98.14%, Val Loss:  1.264, Val Acc: 75.40%, Time: 7.76s 
Epoch: 10, Train Loss: 0.04318, Train Acc: 98.46%, Val Loss:  1.431, Val Acc: 75.02%, Time: 7.51s 
Epoch: 11, Train Loss: 0.03944, Train Acc: 98.62%, Val Loss:  1.352, Val Acc: 76.10%, Time: 7.59s 
Epoch: 12, Train L

Epoch: 12, Train Loss: 0.03805, Train Acc: 98.72%, Val Loss:  2.402, Val Acc: 67.45%, Time: 7.80s *
Epoch: 13, Train Loss: 0.0286, Train Acc: 99.15%, Val Loss:  3.026, Val Acc: 64.85%, Time: 7.62s 
Epoch: 14, Train Loss: 0.02584, Train Acc: 99.10%, Val Loss:  2.549, Val Acc: 64.36%, Time: 7.64s 
Epoch: 15, Train Loss: 0.02812, Train Acc: 99.13%, Val Loss:  2.488, Val Acc: 65.26%, Time: 7.56s 
Epoch: 16, Train Loss: 0.01973, Train Acc: 99.30%, Val Loss:  2.551, Val Acc: 67.05%, Time: 7.71s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.58      0.59      0.58       111
                   Filter       0.57      0.42      0.48       133
    Compute Derived Value       0.57      0.52      0.55       126
            Find Extremum       0.63      0.80      0.71       123
                     Sort       0.53      0.85      0.66       111
          Determin

Epoch: 3, Train Loss: 0.3586, Train Acc: 88.87%, Val Loss:  1.522, Val Acc: 62.05%, Time: 7.06s *
Epoch: 4, Train Loss: 0.2174, Train Acc: 93.10%, Val Loss:  1.705, Val Acc: 61.49%, Time: 7.06s 
Epoch: 5, Train Loss: 0.1497, Train Acc: 95.45%, Val Loss:  1.856, Val Acc: 61.58%, Time: 7.02s 
Epoch: 6, Train Loss: 0.1135, Train Acc: 96.46%, Val Loss:  1.974, Val Acc: 60.69%, Time: 7.12s 
Epoch: 7, Train Loss: 0.08107, Train Acc: 97.37%, Val Loss:  2.197, Val Acc: 59.85%, Time: 6.94s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.50      0.47      0.49       219
                   Filter       0.53      0.49      0.51       215
    Compute Derived Value       0.73      0.41      0.52       256
            Find Extremum       0.76      0.80      0.78       406
                     Sort       0.71      0.79      0.75       126
          Determine Range 

Fold:  4
Epoch: 1, Train Loss:   2.24, Train Acc: 21.25%, Val Loss:  2.016, Val Acc: 29.46%, Time: 9.43s *
Epoch: 2, Train Loss:  1.585, Train Acc: 45.70%, Val Loss:  1.461, Val Acc: 50.77%, Time: 7.48s *
Epoch: 3, Train Loss: 0.9875, Train Acc: 67.07%, Val Loss:  1.365, Val Acc: 59.40%, Time: 7.41s *
Epoch: 4, Train Loss:   0.63, Train Acc: 79.58%, Val Loss:  1.299, Val Acc: 63.72%, Time: 7.42s *
Epoch: 5, Train Loss: 0.4241, Train Acc: 86.85%, Val Loss:  1.324, Val Acc: 65.32%, Time: 7.36s *
Epoch: 6, Train Loss: 0.2928, Train Acc: 90.62%, Val Loss:  1.444, Val Acc: 64.90%, Time: 7.46s 
Epoch: 7, Train Loss: 0.2081, Train Acc: 93.38%, Val Loss:   1.75, Val Acc: 62.60%, Time: 7.45s 
Epoch: 8, Train Loss: 0.1563, Train Acc: 95.24%, Val Loss:  1.773, Val Acc: 63.65%, Time: 7.55s 
Epoch: 9, Train Loss: 0.1569, Train Acc: 95.18%, Val Loss:   1.78, Val Acc: 62.60%, Time: 7.35s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           pr

Epoch: 10, Train Loss: 0.09477, Train Acc: 96.92%, Val Loss:  1.676, Val Acc: 67.45%, Time: 7.56s *
Epoch: 11, Train Loss: 0.08485, Train Acc: 97.32%, Val Loss:  1.767, Val Acc: 65.25%, Time: 7.61s 
Epoch: 12, Train Loss: 0.07682, Train Acc: 97.39%, Val Loss:  1.859, Val Acc: 66.27%, Time: 7.42s 
Epoch: 13, Train Loss: 0.05012, Train Acc: 98.35%, Val Loss:  1.943, Val Acc: 66.42%, Time: 7.44s 
Epoch: 14, Train Loss: 0.04716, Train Acc: 98.47%, Val Loss:    1.9, Val Acc: 65.39%, Time: 7.54s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.56      0.57      0.57        96
                   Filter       0.79      0.57      0.66       191
    Compute Derived Value       0.61      0.80      0.69       173
            Find Extremum       0.45      0.86      0.59       128
                     Sort       0.93      0.61      0.74       140
          Determi

Epoch: 15, Train Loss: 0.03294, Train Acc: 98.98%, Val Loss:  2.232, Val Acc: 65.79%, Time: 7.34s 
Epoch: 16, Train Loss: 0.03678, Train Acc: 98.81%, Val Loss:  2.492, Val Acc: 65.79%, Time: 7.46s 
Epoch: 17, Train Loss: 0.02774, Train Acc: 99.09%, Val Loss:  2.219, Val Acc: 66.93%, Time: 7.36s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.40      0.72      0.51        58
                   Filter       0.40      0.55      0.46       115
    Compute Derived Value       0.66      0.49      0.56       183
            Find Extremum       0.83      0.68      0.75       224
                     Sort       0.90      0.86      0.88       129
          Determine Range       0.73      0.59      0.65       117
Characterize Distribution       0.82      0.71      0.76       174
           Find Anomalies       0.61      0.84      0.71       164
               

Epoch: 1, Train Loss:  1.595, Train Acc: 46.29%, Val Loss:  1.623, Val Acc: 48.77%, Time: 9.25s *
Epoch: 2, Train Loss: 0.7374, Train Acc: 77.10%, Val Loss:   1.36, Val Acc: 59.93%, Time: 7.60s *
Epoch: 3, Train Loss: 0.4053, Train Acc: 87.33%, Val Loss:  1.448, Val Acc: 61.58%, Time: 7.43s *
Epoch: 4, Train Loss: 0.2366, Train Acc: 92.64%, Val Loss:   1.76, Val Acc: 60.34%, Time: 7.65s 
Epoch: 5, Train Loss: 0.1569, Train Acc: 95.25%, Val Loss:  2.081, Val Acc: 59.69%, Time: 7.39s 
Epoch: 6, Train Loss: 0.1176, Train Acc: 96.38%, Val Loss:  2.009, Val Acc: 58.95%, Time: 7.39s 
Epoch: 7, Train Loss: 0.09358, Train Acc: 97.09%, Val Loss:  2.367, Val Acc: 56.81%, Time: 7.57s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.60      0.64      0.62        97
                   Filter       0.39      0.48      0.43       162
    Compute Derived Value      

Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.55      0.48      0.51       244
                   Filter       0.56      0.78      0.66       246
    Compute Derived Value       0.53      0.44      0.48       277
            Find Extremum       0.68      0.69      0.69       275
                     Sort       0.95      0.77      0.85       257
          Determine Range       0.61      0.45      0.52       271
Characterize Distribution       0.73      0.86      0.79       283
           Find Anomalies       0.56      0.69      0.62       300
                  Cluster       0.83      0.77      0.80       271
                Correlate       0.75      0.74      0.75       314

                micro avg       0.67      0.67      0.67      2738
                macro avg       0.68      0.67      0.67      2738
             weighted avg       0.68      0.67      0.67      2738

Confusion Matrix...
[[118

In [ ]:
kf = KFold(n_splits=10)
test_acc_split = []
for split_type,info in split_info.items():
    train_data = dataset_split(info)
    test_acc_split.append(train_split_data(train_data, split_type))

random
14035 41 41
Fold:  1
Epoch: 1, Train Loss:  1.344, Train Acc: 54.23%, Val Loss: 0.6779, Val Acc: 78.13%, Time: 43.17s *
Epoch: 2, Train Loss: 0.4508, Train Acc: 86.06%, Val Loss: 0.4143, Val Acc: 87.11%, Time: 42.28s *
Epoch: 3, Train Loss: 0.2389, Train Acc: 92.39%, Val Loss: 0.3458, Val Acc: 89.67%, Time: 43.00s *
Epoch: 4, Train Loss: 0.1614, Train Acc: 94.75%, Val Loss: 0.2616, Val Acc: 93.23%, Time: 42.45s *
Epoch: 5, Train Loss: 0.1088, Train Acc: 96.51%, Val Loss:  0.295, Val Acc: 91.81%, Time: 42.22s 
Epoch: 6, Train Loss: 0.08015, Train Acc: 97.20%, Val Loss: 0.3082, Val Acc: 92.95%, Time: 42.55s 
Epoch: 7, Train Loss: 0.06802, Train Acc: 97.65%, Val Loss:  0.318, Val Acc: 92.38%, Time: 42.61s 
Epoch: 8, Train Loss: 0.05446, Train Acc: 98.24%, Val Loss:  0.289, Val Acc: 93.09%, Time: 42.85s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value    

Epoch: 1, Train Loss:  1.403, Train Acc: 52.39%, Val Loss: 0.6452, Val Acc: 79.63%, Time: 76.34s *
Epoch: 2, Train Loss: 0.4633, Train Acc: 85.09%, Val Loss: 0.4032, Val Acc: 88.39%, Time: 75.24s *
Epoch: 3, Train Loss: 0.2433, Train Acc: 92.53%, Val Loss: 0.3031, Val Acc: 92.02%, Time: 74.13s *
Epoch: 4, Train Loss: 0.1496, Train Acc: 95.45%, Val Loss: 0.2439, Val Acc: 93.38%, Time: 75.06s *
Epoch: 5, Train Loss: 0.1096, Train Acc: 96.53%, Val Loss:  0.224, Val Acc: 93.66%, Time: 74.34s *
Epoch: 6, Train Loss: 0.08114, Train Acc: 97.41%, Val Loss: 0.2375, Val Acc: 93.66%, Time: 74.67s 
Epoch: 7, Train Loss: 0.07542, Train Acc: 97.46%, Val Loss: 0.2378, Val Acc: 93.95%, Time: 74.57s *
Epoch: 8, Train Loss: 0.05095, Train Acc: 98.28%, Val Loss: 0.1923, Val Acc: 95.09%, Time: 74.45s *
Epoch: 9, Train Loss: 0.0484, Train Acc: 98.36%, Val Loss: 0.2425, Val Acc: 93.95%, Time: 74.78s 
Epoch: 10, Train Loss: 0.03569, Train Acc: 98.78%, Val Loss: 0.2126, Val Acc: 95.01%, Time: 73.20s 
Epoch: 1

Epoch: 1, Train Loss:  1.383, Train Acc: 53.02%, Val Loss: 0.6527, Val Acc: 79.83%, Time: 75.17s *
Epoch: 2, Train Loss: 0.4634, Train Acc: 85.58%, Val Loss: 0.4269, Val Acc: 86.81%, Time: 74.77s *
Epoch: 3, Train Loss: 0.2504, Train Acc: 92.12%, Val Loss:  0.355, Val Acc: 89.45%, Time: 75.19s *
Epoch: 4, Train Loss: 0.1598, Train Acc: 95.01%, Val Loss: 0.2847, Val Acc: 91.73%, Time: 74.35s *
Epoch: 5, Train Loss: 0.1146, Train Acc: 96.30%, Val Loss: 0.2375, Val Acc: 92.80%, Time: 74.81s *
Epoch: 6, Train Loss: 0.08076, Train Acc: 97.33%, Val Loss: 0.2167, Val Acc: 93.23%, Time: 75.04s *
Epoch: 7, Train Loss: 0.06253, Train Acc: 97.94%, Val Loss: 0.2338, Val Acc: 94.01%, Time: 74.98s *
Epoch: 8, Train Loss: 0.05976, Train Acc: 98.05%, Val Loss: 0.2495, Val Acc: 93.51%, Time: 74.71s 
Epoch: 9, Train Loss: 0.05753, Train Acc: 98.05%, Val Loss: 0.2706, Val Acc: 92.94%, Time: 74.50s 
Epoch: 10, Train Loss: 0.03126, Train Acc: 98.95%, Val Loss: 0.2304, Val Acc: 94.37%, Time: 74.65s *
Epoch:

Epoch: 1, Train Loss:  1.359, Train Acc: 53.77%, Val Loss: 0.6785, Val Acc: 77.12%, Time: 76.89s *
Epoch: 2, Train Loss: 0.4668, Train Acc: 85.06%, Val Loss: 0.3305, Val Acc: 88.88%, Time: 73.94s *
Epoch: 3, Train Loss: 0.2429, Train Acc: 92.49%, Val Loss: 0.3025, Val Acc: 90.73%, Time: 74.31s *
Epoch: 4, Train Loss: 0.1571, Train Acc: 95.14%, Val Loss: 0.2228, Val Acc: 93.51%, Time: 74.83s *
Epoch: 5, Train Loss: 0.1034, Train Acc: 96.63%, Val Loss: 0.2158, Val Acc: 94.51%, Time: 75.09s *
Epoch: 6, Train Loss: 0.08234, Train Acc: 97.31%, Val Loss: 0.1872, Val Acc: 94.30%, Time: 74.66s 
Epoch: 7, Train Loss: 0.06591, Train Acc: 97.87%, Val Loss: 0.2464, Val Acc: 93.01%, Time: 74.44s 
Epoch: 8, Train Loss: 0.05658, Train Acc: 98.00%, Val Loss: 0.2755, Val Acc: 92.94%, Time: 73.95s 
Epoch: 9, Train Loss: 0.03826, Train Acc: 98.70%, Val Loss:  0.172, Val Acc: 96.08%, Time: 74.51s *
Epoch: 10, Train Loss: 0.03906, Train Acc: 98.54%, Val Loss:  0.188, Val Acc: 95.58%, Time: 74.61s 
Epoch: 1

Epoch: 4, Train Loss: 0.2107, Train Acc: 93.30%, Val Loss:  1.974, Val Acc: 62.96%, Time: 76.07s 
Epoch: 5, Train Loss: 0.1411, Train Acc: 95.64%, Val Loss:  2.219, Val Acc: 65.02%, Time: 76.50s *
Epoch: 6, Train Loss: 0.1092, Train Acc: 96.74%, Val Loss:  2.291, Val Acc: 63.91%, Time: 76.63s 
Epoch: 7, Train Loss: 0.08563, Train Acc: 97.15%, Val Loss:  2.225, Val Acc: 63.12%, Time: 76.31s 
Epoch: 8, Train Loss: 0.05958, Train Acc: 98.03%, Val Loss:  2.276, Val Acc: 63.99%, Time: 73.76s 
Epoch: 9, Train Loss: 0.04931, Train Acc: 98.40%, Val Loss:  2.738, Val Acc: 60.25%, Time: 77.56s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.73      0.63      0.68       124
                   Filter       0.63      0.51      0.56       130
    Compute Derived Value       0.80      0.17      0.28       116
            Find Extremum       0.47      0.88      0.6

Epoch: 8, Train Loss: 0.06316, Train Acc: 98.08%, Val Loss:  1.965, Val Acc: 65.39%, Time: 77.27s 
Epoch: 9, Train Loss: 0.0469, Train Acc: 98.43%, Val Loss:  2.025, Val Acc: 66.86%, Time: 75.77s *
Epoch: 10, Train Loss: 0.0407, Train Acc: 98.68%, Val Loss:  2.509, Val Acc: 61.96%, Time: 75.90s 
Epoch: 11, Train Loss: 0.03326, Train Acc: 98.88%, Val Loss:  2.204, Val Acc: 64.24%, Time: 76.41s 
Epoch: 12, Train Loss:  0.036, Train Acc: 98.81%, Val Loss:  2.545, Val Acc: 62.69%, Time: 73.39s 
Epoch: 13, Train Loss: 0.03214, Train Acc: 98.85%, Val Loss:  2.392, Val Acc: 64.33%, Time: 76.36s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.50      0.67      0.58       123
                   Filter       0.46      0.53      0.50       124
    Compute Derived Value       0.63      0.31      0.41       130
            Find Extremum       0.70      0.89     